In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as pl
import cPickle as pkl
import colormaps as mycmps
import signale.tools as tools
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import scipy.signal as scsig
import signale

In [4]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/'

In [35]:
def ncs_loader(fileName):
    ncs_dtype = np.dtype([
        ('qTimeStamps'  ,     '<u8'),
        ('ChannelNumber'  , '<u4'),
        ('SampleFreq',      '<u4'),
        ('NumValidSamples', '<u4'),
        ('Samples'  ,       '<i2',(512,)),
    ])
    return np.memmap(fileName,dtype=ncs_dtype,offset=(16 * 2**10))
def zScore(array):
    return(array - np.average(array))/ np.std(array)
def RippTrigeredPopRate(lfpPath,Tfolder,rippDB):
    rDB = pd.read_pickle(rippDB)
    tpaths = sorted([os.path.join(tFile[0],tFile[1]) for tFile in tools.locate('pl*.t',Tfolder)])
    lfp = pkl.load(open(lfpPath))
    bins = np.arange(np.floor(lfp.timeAxis[0]),np.floor(lfp.timeAxis[-1])+1,10)
    r = np.zeros(bins.size-1)
    binC = bins[:-1]+5
    for tFile in tpaths:
        t = signale.io.load_tFile(tFile)
        r += np.histogram(t.spike_times,bins)[0]
    r = zScore(r)
    rDB = pd.read_pickle(rippDB)
    lfpFile = lfpPath.split('/')[-1]
    session = lfpPath.split('/')[-2]
    session = session.split('p')[0]+'p0'+session.split('p')[1]
    date = lfpPath.split('_')[-2]
    relevantRipples = rDB[rDB.date==date]
    relevantRipples = relevantRipples[relevantRipples.session==session]
    #Rcond = np.logical_and(relevantRipples.c ==1, relevantRipples.wc == 1)
    #Gcond = np.logical_and(relevantRipples.c ==2, relevantRipples.wc == 2)
    Rcond = np.logical_or(relevantRipples.wc ==5, relevantRipples.wc == 8)
    Gcond = (relevantRipples.wc==6)
    tsR = []
    tsG = []
    for t_p in relevantRipples.t_peak[Rcond]:
        idx,tStamp = tools.findNearest(binC,t_p)
        tsR.append(r[idx-5:idx+5])
    for t_p in relevantRipples.t_peak[Gcond]:
        idx,tStamp = tools.findNearest(binC,t_p)
        tsG.append(r[idx-5:idx+5])
    return tsR,tsG

In [6]:
lfpPaths = [os.path.join(lfp[0],lfp[1]) for lfp in tools.locate('*p*.lfp',animalPath)         ]
tFolders = ['/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_05-31-16_Day1/parrentData/',
            '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-01-16_Day2/parrentData/',
            '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-02-16_Day3/parrentData/',
            '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-03-16_Day4/parrentData/'
           ]
DBpath = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3906Control/dataFiles/sleepDB/RipplesTimeDB.pd'

In [36]:
rr = []
gg = []
for ii,item in enumerate(lfpPaths[:]):
    r,g = RippTrigeredPopRate(item,tFolders[ii/2],DBpath)
    rr.extend(r)
    gg.extend(g)

In [39]:
sns.tsplot(rr[:])
sns.tsplot(gg[:16000],color='r')

In [127]:
pkl.dump(rr,open('/mnt/Data/ephysdata/Gerbils/popRateRipp.npArr','wb'),pkl.HIGHEST_PROTOCOL)
pkl.dump(gg,open('/mnt/Data/ephysdata/Gerbils/popRateFGB.npArr','wb'),pkl.HIGHEST_PROTOCOL)

In [12]:
rDB = pd.read_pickle(DBpath)

In [34]:
pl.hist(rDB[rDB.date=='06-01-16'].wc,alpha=0.2);

In [19]:
rDB

,t_i,t_f,peak_value,t_peak,epoch,date,session,animal,PC1,PC2,PC3,PC4,PC5,wPC1,wPC2,wPC3,wPC4,wPC5,wc,c
0,943,970,28358.1,958,rem,05-31-16,01-sleep01,Rat3906,-3.194073,-1.347406,-0.911356,0.578150,-0.310869,2.259520,-2.927950,0.186095,0.499981,-1.043956,6,3
1,2946,3010.5,29915.9,2997.5,sws,05-31-16,01-sleep01,Rat3906,-0.016944,-1.504324,0.951234,0.627511,1.251902,1.400439,-1.691174,0.343242,-0.918347,-0.326867,6,3
2,5981.5,6040,29056.7,6022.5,rem,05-31-16,01-sleep01,Rat3906,0.294672,-1.607320,-1.958373,0.568061,0.694380,0.361065,-1.821026,3.678320,0.709114,-0.617302,6,3
3,7765.5,7811.5,29640.9,7777.5,rem,05-31-16,01-sleep01,Rat3906,-1.055796,-0.332559,2.829267,-0.922702,1.453755,1.691479,-0.364842,-3.022505,1.067924,-0.708458,6,3
4,8890,8912,29356.1,8901,rem,05-31-16,01-sleep01,Rat3906,-3.605473,-0.723723,-1.918700,0.668426,-3.627091,1.306111,-0.324896,0.237165,0.321808,-0.806000,6,3
5,9200,9240.5,30479.3,9224.5,rem,05-31-16,01-sleep01,Rat3906,-3.174977,1.193918,-1.157637,-1.402485,0.179235,0.712532,-0.246301,-1.173250,2.163899,-0.018717,6,3
6,10351.5,10397,30642.6,10383.5,rem,05-31-16,01-sleep01,Rat3906,1.591425,0.371593,2.160409,-0.437236,-0.083214,-1.966900,0.004414,-2.356174,-1.492565,1.326766,6,3
7,10516.5,10545,31566.8,10533,rem,05-31-16,01-sleep01,Rat3906,1.203106,-1.678823,0.923801,-1.417761,-0.866647,-0.624038,-1.954741,0.046263,-0.710093,2.052844,6,3
8,12501.5,12558,31448.8,12545.5,rem,05-31-16,01-sleep01,Rat3906,-1.082038,-0.210192,0.794521,-1.628886,0.576896,2.280643,-0.448315,-0.495838,0.303290,2.000109,6,3
9,13176.5,13209,34086.1,13188,rem,05-31-16,01-sleep01,Rat3906,-1.668084,-1.414450,0.246376,0.054049,1.328269,2.920739,-1.709465,0.475638,0.574515,-0.568963,6,3
